In [1]:
import cv2
import numpy as np
import os
import copy
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from sklearn.cluster import MeanShift
from random import randint
#from detector import Detectors
from scipy.spatial import distance as dist

In [2]:
class Entity(object):
    def __init__(self, vid, windows, frame):
        self.vid = vid
        self.windows = windows
        self.center = self._set_center(windows)
        self.trajectory = [self.center]
        self.tracker = self._init_tracker(windows, frame)

    def _set_center(self, windows):
        x, y, w, h = windows
        x = (2 * x + w) / 2
        y = (2 * y + h) / 2
        center = np.array([np.float32(x), np.float32(y)], np.float32)
        return center

    def _init_tracker(self, windows, frame):
        x, y, w, h = windows
        track_types={"BOOSTING":cv2.TrackerBoosting_create, "MIL":cv2.TrackerBoosting_create, "KCF":cv2.TrackerBoosting_create, "TLD":cv2.TrackerBoosting_create, "MEDIANFLOW":cv2.TrackerBoosting_create, "GOTURN":cv2.TrackerBoosting_create, "MOSSE":cv2.TrackerBoosting_create, "CSRT":cv2.TrackerBoosting_create}
        #multitracker=track_types['KCF']()
        tracker = track_types['CSRT']()
        tracker.init(frame, (x, y, w, h))
        return tracker

    def update(self, frame):
        self.tracker.update(frame)
        ok, new_box = self.tracker.update(frame)
        if ok:
            x, y, w, h = int(new_box[0]), int(new_box[1]), int(new_box[2]), int(new_box[3])
            self.center = self._set_center((x, y, w, h))
            self.windows = (x, y, w, h)
            self.trajectory.append(self.center)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
            #cv2.putText(frame, "cell", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.polylines(frame, [np.int32(self.trajectory)], 0, (0, 0, 255))

In [3]:
def gray_img(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [4]:
def normalize(image):
    img = image.copy().astype(np.float32)
    img -= np.mean(img)
    img /= np.linalg.norm(img)
    img = np.clip(img, 0, 255)
    img *= (1./float(img.max()))
    return (img*255).astype(np.uint8)

In [5]:
def read_img(path):
    sequence_tif = {}
    gary_seq_tif = {}
    for root, dirs, files, in os.walk(path):
        dirs.sort()
        for dir in dirs:
            sequence_tif[dir] = []
            gary_seq_tif[dir] = []
            for _, _, files_1 in os.walk(os.path.join(path, dir)):
                files_1.sort()
                for file in files_1:
                    new_dir_path = os.path.join(path, dir)
                    temp_img = cv2.imread(os.path.join(new_dir_path, file))
                    gray_imgs = gray_img(temp_img.copy())
                    sequence_tif[dir].append(temp_img)
                    gary_seq_tif[dir].append(gray_imgs)

    return sequence_tif,gary_seq_tif

In [6]:
#ass1
def max_filter(img,N,m):
    imgA=np.zeros_like(img)
    #imgA=img.copy()
    padding_img=cv2.copyMakeBorder(img,m,m,m,m,cv2.BORDER_CONSTANT)
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            new=padding_img[r:r+N,c:c+N].copy()
            #new=new*mask
            minval,maxval,minindex,maxindex=cv2.minMaxLoc(new)
            imgA[r,c]=maxval
    return imgA
def min_filter(img,N,m):
    imgB=np.zeros_like(img)
    #imgB=img.copy()
    padding_img=cv2.copyMakeBorder(img,m,m,m,m,cv2.BORDER_CONSTANT)
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            new=padding_img[r:r+N,c:c+N].copy()
            #new=new*mask
            minval,maxval,minindex,maxindex=cv2.minMaxLoc(new)
            imgB[r,c] = minval
    return imgB
def subtract_img(img1,img2):
    img_O=np.zeros_like(img1)
    for r in range(img1.shape[0]):
        for c in range(img1.shape[1]):
            img_O[r,c]=img1[r,c].astype('int32')-img2[r,c].astype('int32')
            #if img_O[r,c]<=0:
                #img_O[r,c]+=255
        
    min=np.min(img_O)
    max=np.max(img_O)
    for r in range(img_O.shape[0]):
        for c in range(img_O.shape[1]):
            img_O[r,c]=(img_O[r,c]-min)*(255/(max-min))
        
    img_O=img_O.astype(np.uint8)
    return img_O

In [7]:
def overlap(box1, box2):
    
    endx = max(box1[0] + box1[2], box2[0] + box2[2])
    startx = min(box1[0], box2[0])
    width = box1[2] + box2[2] - (endx - startx)

    endy = max(box1[1] + box1[3], box2[1] + box2[3])
    starty = min(box1[1], box2[1])
    height = box1[3] + box2[3] - (endy - starty)

    if (width <= 0 or height <= 0):
        return 0
    else:
        Area = width * height
        Area1 = box1[2] * box1[3]
        Area2 = box2[2] * box2[3]
        ratio = Area / (Area1 + Area2 - Area)

        return ratio

In [8]:
def tracking_cells(sequence,base):
    track_types={"BOOSTING":cv2.TrackerBoosting_create, "MIL":cv2.TrackerBoosting_create, "KCF":cv2.TrackerBoosting_create, "TLD":cv2.TrackerBoosting_create, "MEDIANFLOW":cv2.TrackerBoosting_create, "GOTURN":cv2.TrackerBoosting_create, "MOSSE":cv2.TrackerBoosting_create, "CSRT":cv2.TrackerBoosting_create}
    multitracker=track_types['KCF']()
    track_list = []
    counter = 0
    N=27
    m=N//2
    for k in range(len(sequence)):
            frame=copy.copy(sequence[k])
            g=gray_img(frame)
            y_size = frame.shape[0]
            x_size = frame.shape[1] 
            '''
            imgA=min_filter(g,N,m)
            imgB=max_filter(imgA,N,m)
            imgO=subtract_img(frame,imgB)
            imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
            ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            '''
            imgO=normalize(frame)
            imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
            ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
            d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
            contours1=[] 
            centers=[]
            for i in contours:
                #x, y, w, h = cv2.boundingRect(i)
                if cv2.contourArea(i)>25:  #Remove small connected areas
                    #x, y, w, h = cv2.boundingRect(i)
                    contours1.append(i)
                    for m,n in zip(contours1,range(len(contours1))):
                        M = cv2.moments(m)
                        cX=int(M["m10"]/M["m00"])
                        cY=int(M["m01"]/M["m00"])
                        centers.append((cX,cY))
                    for t in contours1:
                        x, y, w, h = cv2.boundingRect(t)

                    e = Entity(counter, (x, y, w, h), frame)

                    # Exclude existing targets in the tracking list
                    if track_list:
                        count = 0
                        num = len(track_list)
                        for p in track_list:
                            if overlap((x, y, w, h), p.windows) < base:
                                count += 1
                        if count == num:
                            track_list.append(e)
                    else:
                        track_list.append(e)
                    counter += 1
            if track_list:
                tlist = copy.copy(track_list)
                for e in tlist:
                    x, y = e.center
                    #if 1 < x < x_size  and 1 < y < y_size :
                    #or
                    #if 1 < x < x_size -1 and 1 < y < y_size - 1:
                    if 1 < x < x_size and 1 < y < y_size :
                        e.update(frame)
                    else:
                        track_list.remove(e)
            plt.imsave('trackingc_'+str(k)+'.jpg',frame)

In [9]:
path='/Users/admin/Downloads/COMP9517 20T2 Group Project Image Sequences-2/'

In [ ]:
path2=path+'PhC-C2DL-PSC'
original_img,grays=read_img(path2)
base=0.2
#base=2
tracking_cells(original_img['Sequence 1'],base)